# Pipeline to manage anomalies processing

Before using any dataset, it is necessary to clean the data to obtain a clear, formatted, and complete dataset. This tedious step is the first step in data analysis.

In this notebook, there are building blocks (functions) that can be used later to perform the identified preprocessing steps.

- Remove missing values
- Remove duplicates
- Remove special characters
- Convert numbers to letters
- Identify the language of the review and translate it if necessary
- Correct spelling errors

In [87]:
import polars as pl
import re
from num2words import num2words
import langid
import concurrent
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
from deep_translator import GoogleTranslator
from spellchecker import SpellChecker
from symspellpy.symspellpy import SymSpell, Verbosity
import os
from dotenv import load_dotenv
import logging

In [88]:
# Simple loger for pipeline execution
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

logging.getLogger("langid").setLevel(logging.WARNING)
logging.getLogger("transformers").setLevel(logging.WARNING)

In [89]:
# Global variables
load_dotenv(dotenv_path="../.env")
NUM_THREAD = int(os.environ.get("NUM_THREADS"))
logger.info(f"NUM_THREAD fixed to {NUM_THREAD}")

2025-11-18 12:43:18,625 - INFO - NUM_THREAD fixed to 12


## Remove missing values

In [90]:
def clean_missing_values(df: pl.DataFrame, column_name: str) -> tuple[pl.DataFrame, int]:
    """
    Remove rows from a DataFrame where the specified column has missing values,
    and return the cleaned DataFrame and the number of missing values removed.
    
    Args:
        df (pl.DataFrame): The DataFrame to clean.
        column_name (str): Column to check for missing values.
        
    Returns:
        tuple[pl.DataFrame, int]: Cleaned DataFrame and number of missing values removed.
    """
    
    # Drop rows where the specified column is null
    df_clean = df.drop_nulls(subset=[column_name])

    nb_missing_values = df[column_name].null_count()
    
    return df_clean,nb_missing_values

## Remove duplicates

In [91]:
def remove_duplicates(df: pl.DataFrame, subset_columns: list) -> tuple[pl.DataFrame, int]:
    """
    Remove duplicate rows from a DataFrame based on specified columns,
    and return the cleaned DataFrame and the number of duplicates removed.
    
    Args:
        df (pl.DataFrame): The DataFrame to clean.
        subset_columns (list): List of columns to consider for duplicates.
        
    Returns:
        tuple[pl.DataFrame, int]: DataFrame with duplicates removed and number of duplicates found.
    """
    # Nombre de lignes avant suppression
    initial_count = df.height

    # Supprimer les doublons selon les colonnes spécifiées
    df_clean = df.unique(subset=subset_columns)

    # Nombre de doublons supprimés
    nb_duplicates = initial_count - df_clean.height

    return df_clean, nb_duplicates

## Remove spacial characters

In [92]:
# def remove_special_characters(df: pl.DataFrame, column_name: str) -> pl.DataFrame:
#     """
#     Remove special characters from a specified text column using regex.

#     Args:
#         df (pl.DataFrame): Input Polars DataFrame.
#         column_name (str): Name of the text column to clean.
#         keep (str): Optional string of characters to preserve (e.g., ".," to keep dots and commas).

#     Returns:
#         pl.DataFrame: New DataFrame with cleaned text in the specified column.
#     """

#     keep = "'’"

#     # Pattern to remove emails
#     email_pattern = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b"

#     # Pattern to remove unwanted characters
#     chars_pattern = rf"[^\w\s{re.escape(keep)}]"

#     # Pattern to remove hashtags (#word)
#     hashtag_pattern = r"#\S+"

#     # Apostrophe
#     apostrophe_pattern = r"(?<!n)'|'(?!t)"

#     # Pattern to replace apostrophes with a space
#     #apostrophe_pattern = r"(?<![A-Za-z])[’']|[’'](?![A-Za-z])"

#     # Pattern to remove url
#     url_pattern = r"https?://\S+|www\.\S+"

   
#     def clean_text(text):
#         # 1. Remove emails
#         text = re.sub(email_pattern, "", text)
#         # 2. Remove URLs
#         text = re.sub(url_pattern, "", text)
#         # 3. Remove hashtags
#         text = re.sub(hashtag_pattern, "", text)
#         # 4. Keep apostrophes between two letters
#         text = re.sub(apostrophe_pattern, " ", text)
#         # 4.1 Replace slashes with spaces
#         text = text.replace("/", " ")
#         # 4.1 Replace hyphen with spaces
#         text = text.replace("-", " ")
#         # 5. Remove unwanted characters
#         text = re.sub(chars_pattern, "", text)
#         # 6. Clean multiple spaces
#         text = re.sub(r"\s+", " ", text).strip()
#         return text

#     df_cleaned = df.with_columns(
#         pl.col(column_name).map_elements(clean_text).alias(column_name)
#     )

#     return df_cleaned

In [93]:
# Liste des contractions courantes à conserver
CONTRACTIONS = [
    "aren't","can't","couldn't","didn't","doesn't","don't","hadn't","hasn't",
    "haven't","he's","I'm","I've","isn't","it's","she's",
    "shouldn't","that's","there's","they're","they've",
    "we're","we've","weren't","what's","where's","who's","won't","wouldn't","you're","you've",

    "aren’t","can’t","couldn’t","didn’t","doesn’t","don’t","hadn’t","hasn’t",
    "haven’t","he’s","I’m","I’ve","isn’t","it’s","she’s",
    "shouldn’t","that’s","there’s","they’re","they’ve",
    "we’re","we’ve","weren’t","what’s","where’s","who’s","won’t","wouldn’t","you’re","you’ve"
]

CONTRACTIONS_PATTERN = re.compile(
    r'\b(' + '|'.join(re.escape(c) for c in CONTRACTIONS) + r')\b', re.IGNORECASE
)

def remove_special_characters(df: pl.DataFrame, column_name: str, keep: str = "") -> pl.DataFrame:
    """
    Clean text column while preserving English contractions,
    removing possessive apostrophes, emails, URLs, hashtags, unwanted chars,
    replacing / and - with space.
    """

    # Emails, URLs, hashtags
    email_pattern = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b"
    url_pattern = r"https?://\S+|www\.\S+"
    hashtag_pattern = r"#\S+"

    # Possessive apostrophes ('s or ’s)
    possessive_pattern = r"(\w)[’']s\b"

    # Other apostrophes (to be removed) except contractions
    apostrophe_pattern = r"[’']"

    # Unwanted chars except letters, digits, spaces, keep chars, and '
    chars_pattern = rf"[^\w\s{re.escape(keep)}']"

    def clean_text(text):
        if not isinstance(text, str):
            return text

        # Preserve contractions
        contractions_map = {}
        def replace_contraction(m):
            key = f"__CONTRACTION_{len(contractions_map)}__"
            contractions_map[key] = m.group(0)
            return key

        text = CONTRACTIONS_PATTERN.sub(replace_contraction, text)

        # 1. Remove emails, URLs, hashtags
        text = re.sub(email_pattern, "", text)
        text = re.sub(url_pattern, "", text)
        text = re.sub(hashtag_pattern, "", text)

        # 2. Remove possessive apostrophes
        text = re.sub(possessive_pattern, r"\1 s", text)

        # 3. Remove remaining apostrophes
        text = re.sub(apostrophe_pattern, " ", text)

        # 4. Replace / and - with spaces
        text = text.replace("/", " ").replace("-", " ")

        # 5. Remove unwanted characters
        text = re.sub(chars_pattern, " ", text)

        # 6. Clean multiple spaces
        text = re.sub(r"\s+", " ", text).strip()

        # 7. Restore contractions
        for k, v in contractions_map.items():
            text = text.replace(k, v)

        # HTML artefacts
        text = re.sub(r"strong\s+text", "", text, flags=re.IGNORECASE)

        # with abreviation
        text = re.sub(r"\bw\b", "with", text, flags=re.IGNORECASE)

        return text

    df_cleaned = df.with_columns(
        pl.col(column_name).map_elements(clean_text).alias(column_name)
    )
    return df_cleaned


In [94]:
# Test

df_test_complet = pl.DataFrame({
    "text": [
        # Emails / URLs
        "Contact me at test@example.com or visit https://example.com!",
        "Check www.site.com/path or mail me at user.name+tag@domain.co.uk",
        
        # Hashtags
        "Learning #Python3 and #DataScience is fun!",

        # Contractions à garder
        "I'm happy, it's amazing, don't worry, he can't come, she's here, we've done it. Youpi!Help  ? LOL",
        
        # Possessifs à transformer
        "John's book, teacher's notes, O'Reilly's guide, my friend's dog.",

        # Apostrophes isolées
        "This ' is a lone apostrophe, 'alone' also.",

        # Slashes / tirets
        "Hello/world, well-known concepts, 12/05/2024, a/b/c",
        
        # Ponctuation à garder
        "Keep commas, periods. And exclamations!",

        # Accents et Unicode
        "Café, résumé, niño, façade, naïve, coöperate, 😊 emoji,©symbol hey,ho",

        # Combinaison complexe
        "She's John's friend's dog/cat, don't forget! Visit www.site.com/path.",

        "She'd be happy.I'll be there in a minute but i wouldn't do this if i were you",

        "strong textDoes accessibleGO have ANY option for airline travel First Class",

        "It's I've I'm",
        "I’m traveling to Maui in late July, does anyone have recommendations on hotels, etc that have handicap accessible pools? I use a scooter/walker, can not do steps/stairs very well without my braces (aren’t water proof lol).",
        "Anyone w private pool ?",
        "Tim timshe52@Gmail.com"
    ]
})


# --- Test de la fonction ---
df_cleaned = remove_special_characters(df_test_complet, "text")
# Afficher toutes les lignes
for i, row in enumerate(df_cleaned["text"]):
    print(f"{i}: {row}")

0: Contact me at or visit
1: Check or mail me at
2: Learning and is fun
3: I'm happy it's amazing don't worry he can't come she's here we've done it Youpi Help LOL
4: John s book teacher s notes O Reilly s guide my friend s dog
5: This is a lone apostrophe alone also
6: Hello world well known concepts 12 05 2024 a b c
7: Keep commas periods And exclamations
8: Café résumé niño façade naïve coöperate emoji symbol hey ho
9: She's John s friend s dog cat don't forget Visit
10: She d be happy I ll be there in a minute but i wouldn't do this if i were you
11: Does accessibleGO have ANY option for airline travel First Class
12: It's I've I'm
13: I’m traveling to Maui in late July does anyone have recommendations on hotels etc that have handicap accessible pools I use a scooter walker can not do steps stairs very well without my braces aren’t water proof lol
14: Anyone with private pool
15: Tim


## Convert numbers to letters

In [95]:
def numbers_to_words(df: pl.DataFrame, column_name: str) -> pl.DataFrame:
    """
    Convert all numbers in a text column into words using num2words.

    Args:
        df (pl.DataFrame): Input DataFrame.
        column_name (str): Name of the text column to process.
        lang (str): Language code (e.g., 'en' or 'fr').

    Returns:
        pl.DataFrame: New DataFrame with numbers replaced by words.
    """
    def convert_numbers(text: str) -> str:
        if not isinstance(text, str) or not text.strip():
            return text

        def replace_number(m):
            num_text = num2words(int(m.group()))
            left = ' ' if m.start() > 0 and text[m.start()-1].isalnum() else ''
            right = ' ' if m.end() < len(text) and text[m.end():m.end()+1].isalnum() else ''
            return f"{left}{num_text}{right}"

        return re.sub(r'\d+', replace_number, text)

    df_converted = df.with_columns(
        pl.col(column_name).map_elements(convert_numbers).alias(column_name)
    )

    return df_converted

## Languages and translation

### Languages detection

In [96]:
def detect_language_parallel(df: pl.DataFrame, column_name: str, num_threads: int = 4) -> tuple[pl.DataFrame, int]:
    """
    Detect the language of a text column in a Polars DataFrame using langid in parallel.

    Args:
        df (pl.DataFrame): Input DataFrame.
        column_name (str): Name of the text column to process.
        num_threads (int): Number of threads to use for parallel processing (default=4).

    Returns:
        tuple[pl.DataFrame, int]: 
            - DataFrame with an added column 'detected_lang' containing language codes.
            - Number of sentences not detected as English ('en').
    """
    def detect_lang(text: str) -> str:
        """Return the language code of a single text using langid."""
        if not isinstance(text, str) or not text.strip():
            return None
        lang, _ = langid.classify(text)
        return lang

    # Convert the Polars column to a Python list
    texts = df[column_name].to_list()

    # Parallel processing with ThreadPoolExecutor
    all_langs = []
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        for result in tqdm(executor.map(detect_lang, texts), total=len(texts), desc="Language detection"):
            all_langs.append(result)

    # Return new DataFrame with added column
    df_result = df.with_columns(pl.Series("detected_lang", all_langs))

    nb_non_english = sum(lang != "en" for lang in all_langs if lang is not None)

    return df_result,nb_non_english

### Translation in english

In [97]:
def translate_non_english_threadsafe(df: pl.DataFrame,
                                     column_name: str,
                                     detected_lang_col: str = "detected_lang",
                                     num_threads: int = 4) -> pl.DataFrame:
    """
    Thread-safe translation: one translator per thread.
    """
    def translate_one(text: str) -> str:
        if not isinstance(text, str) or not text.strip():
            return text
        try:
            translator = GoogleTranslator(source='auto', target='en')  # local traductor
            return translator.translate(text)
        except Exception as e:
            return f"[ERROR: {e}]"

    indices_to_translate = []
    texts_to_translate = []
    for i, (text, lang) in enumerate(zip(df[column_name].to_list(), df[detected_lang_col].to_list())):
        if lang != 'en':
            indices_to_translate.append(i)
            texts_to_translate.append(text)

    translated_texts = df[column_name].to_list()

    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        for idx, translation in zip(
            indices_to_translate,
            tqdm(executor.map(translate_one, texts_to_translate),
                 total=len(texts_to_translate),
                 desc="Translating non-English reviews")
        ):
            translated_texts[idx] = translation

    return df.with_columns(pl.Series(column_name, translated_texts))

## Correction of spelling error

In [98]:
def correct_spelling_symspell(
    df: pl.DataFrame,
    column_name: str,
    batch_size: int = 10000,
    n_threads: int = 4,
    max_edit_distance: int = 2,
    dictionary_path: str = "frequency_dictionary_en_82_765.txt"
) -> pl.DataFrame:
    """
    Correct spelling using SymSpell (fast) on a Polars DataFrame text column.
    Supports batching and multithreading.

    Args:
        df: Polars DataFrame
        column_name: Name of the text column
        batch_size: Number of rows per batch
        n_threads: Number of parallel threads
        max_edit_distance: Max edit distance for corrections
        dictionary_path: Path to SymSpell frequency dictionary

    Returns:
        Polars DataFrame with corrected text
    """

    # Initialize SymSpell 
    sym_spell = SymSpell(max_dictionary_edit_distance=max_edit_distance, prefix_length=7)
    if not os.path.exists(dictionary_path):
        raise FileNotFoundError(f"Dictionary not found: {dictionary_path}")
    sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

    # Function to correct a single review 
    def fix_text(text: str) -> str:
        if not isinstance(text, str) or not text.strip():
            return text
        corrected_words = []
        for word in text.split():
            suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance)
            corrected_words.append(suggestions[0].term if suggestions else word)
        return " ".join(corrected_words)

    # Split dataframe into batches
    batches = [df.slice(i, batch_size) for i in range(0, len(df), batch_size)]
    corrected_reviews = []

    # Parallel batch processing 
    with concurrent.futures.ThreadPoolExecutor(max_workers=n_threads) as executor:
        for batch_result in tqdm(
            executor.map(lambda b: [fix_text(t) for t in b[column_name]], batches),
            total=len(batches),
            desc="Spell-checking with SymSpell"
        ):
            corrected_reviews.extend(batch_result)

    # --- Return DataFrame with corrected column ---
    df_corrected = df.with_columns(
        pl.Series(name=column_name, values=corrected_reviews)
    )

    return df_corrected

## Main

This script automatically runs the entire pipeline on a .csv file and saves the result.

In [99]:
def preprocess_pipeline(input_csv: str, column_name: str, output_csv: str):
    """
    Apply the full preprocessing pipeline to the given CSV file.
    """
    df = pl.read_csv(input_csv)
    logger.info(f"DataFrame {os.path.splitext(os.path.basename(input_csv))[0]} loaded : {df.shape[0]} rows x {df.shape[1]} columns")
    df,nb_missing_values = clean_missing_values(df, column_name)
    logger.info(f"{nb_missing_values} missing reviews detected and cleaned.")
    df,nb_duplicates = remove_duplicates(df, column_name)
    logger.info(f"{nb_duplicates} duplicated reviews detected and cleaned.")
    df = numbers_to_words(df, column_name)
    logger.info(f"Numerical numbers converted to string numbers.")
    df = remove_special_characters(df, column_name)
    logger.info(f" Special characters removed.")
    df,nb_to_translate = detect_language_parallel(df, column_name, NUM_THREAD)
    logger.info(f"{nb_to_translate} reviews are potentially not in english.")
    df = translate_non_english_threadsafe(df, column_name, "detected_lang", NUM_THREAD)
    logger.info(f"{nb_to_translate} have been translated in english.")
    # df = correct_spelling_symspell(
    #     df,
    #     column_name=column_name,
    #     batch_size=10000,
    #     n_threads=NUM_THREAD,
    #     max_edit_distance=2,
    #     dictionary_path="../data/original/frequency_dictionary_en_82_765.txt"
    # )
    df.write_csv(output_csv)
    logger.info(f"Cleaned Dataframe saved at {output_csv}")

In [ ]:
from pathlib import Path

# Dossiers
input_folder = Path("../data/original/dataset")
output_folder = Path("../data/processed/data_clean")
output_folder.mkdir(parents=True, exist_ok=True)  # créer le dossier si besoin

# Lancer le pipeline sur tous les fichiers CSV
for file_path in input_folder.glob("*.csv"):
    # Nom de fichier sans extension
    stem = file_path.stem  # ex: data_airline_reviews_1
    
    # Choix de la colonne à nettoyer
    column_name = "review"
    if "yelp" in stem.lower():
        column_name = "text"
    
    output_file = output_folder / f"{stem}_cleaned.csv"
    
    preprocess_pipeline(str(file_path), column_name, str(output_file))


2025-11-18 12:43:18,752 - INFO - DataFrame data_accessiblego loaded : 2225 rows x 4 columns
2025-11-18 12:43:18,752 - INFO - 0 missing reviews detected and cleaned.
2025-11-18 12:43:18,763 - INFO - 329 duplicated reviews detected and cleaned.
2025-11-18 12:43:18,819 - INFO - Numerical numbers converted to string numbers.


Processing data_accessiblego.csv -> data_accessiblego_cleaned.csv | column: review


2025-11-18 12:43:19,254 - INFO -  Special characters removed.
Language detection: 100%|██████████| 1896/1896 [00:00<00:00, 2242.30it/s]
2025-11-18 12:43:20,885 - INFO - 8 reviews are potentially not in english.
Translating non-English reviews:  92%|█████████▏| 12/13 [00:00<00:00, 13.77it/s]
2025-11-18 12:43:21,782 - INFO - 8 have been translated in english.
2025-11-18 12:43:21,786 - INFO - Cleaned Dataframe saved at ..\data\processed\data_clean\data_accessiblego_cleaned.csv
2025-11-18 12:43:21,817 - INFO - DataFrame data_activities_reviews loaded : 46377 rows x 4 columns
2025-11-18 12:43:21,817 - INFO - 0 missing reviews detected and cleaned.
2025-11-18 12:43:21,828 - INFO - 503 duplicated reviews detected and cleaned.


Processing data_activities_reviews.csv -> data_activities_reviews_cleaned.csv | column: review


2025-11-18 12:43:22,225 - INFO - Numerical numbers converted to string numbers.
2025-11-18 12:43:26,160 - INFO -  Special characters removed.
Language detection: 100%|██████████| 45874/45874 [00:35<00:00, 1310.37it/s]
2025-11-18 12:44:02,531 - INFO - 3770 reviews are potentially not in english.
Translating non-English reviews: 100%|█████████▉| 3770/3771 [02:06<00:00, 29.92it/s]
2025-11-18 12:46:08,618 - INFO - 3770 have been translated in english.
2025-11-18 12:46:08,626 - INFO - Cleaned Dataframe saved at ..\data\processed\data_clean\data_activities_reviews_cleaned.csv
2025-11-18 12:46:08,638 - INFO - DataFrame data_airline_reviews_1 loaded : 3701 rows x 20 columns
2025-11-18 12:46:08,648 - INFO - 0 missing reviews detected and cleaned.
2025-11-18 12:46:08,652 - INFO - 9 duplicated reviews detected and cleaned.


...

2025-11-18 13:28:22,412 - INFO - DataFrame data_tripadvisor_hotel_reviews loaded : 878561 rows x 10 columns
2025-11-18 13:28:22,419 - INFO - 0 missing reviews detected and cleaned.
2025-11-18 13:28:22,500 - INFO - 988 duplicated reviews detected and cleaned.
2025-11-18 13:28:44,842 - INFO - Numerical numbers converted to string numbers.
2025-11-18 13:31:47,397 - INFO -  Special characters removed.
Language detection: 100%|██████████| 877573/877573 [13:26<00:00, 1088.62it/s]  
2025-11-18 13:48:16,583 - INFO - 106928 reviews are potentially not in english.
Translating non-English reviews: 100%|█████████▉| 106928/106929 [2:21:17<00:00, 12.61it/s]  
2025-11-18 16:09:46,253 - INFO - 106928 have been translated in english.
2025-11-18 16:09:50,220 - INFO - Cleaned Dataframe saved at ..\data\processed\data_clean\data_tripadvisor_hotel_reviews_cleaned.csv


Processing data_twitter.csv -> data_twitter_cleaned.csv | column: review


2025-11-18 16:09:51,742 - INFO - DataFrame data_twitter loaded : 10000 rows x 6 columns
2025-11-18 16:09:51,789 - INFO - 0 missing reviews detected and cleaned.
2025-11-18 16:09:51,820 - INFO - 0 duplicated reviews detected and cleaned.
2025-11-18 16:09:52,146 - INFO - Numerical numbers converted to string numbers.
2025-11-18 16:09:58,525 - INFO -  Special characters removed.
Language detection: 100%|██████████| 10000/10000 [00:39<00:00, 254.16it/s] 
2025-11-18 16:10:49,866 - INFO - 6 reviews are potentially not in english.
Translating non-English reviews:  83%|████████▎ | 5/6 [00:01<00:00,  4.23it/s]
2025-11-18 16:10:51,141 - INFO - 6 have been translated in english.
2025-11-18 16:10:51,165 - INFO - Cleaned Dataframe saved at ..\data\processed\data_clean\data_twitter_cleaned.csv


Processing data_yelp_reviews.csv -> data_yelp_reviews_cleaned.csv | column: text


2025-11-18 16:11:14,004 - INFO - DataFrame data_yelp_reviews loaded : 6990280 rows x 9 columns
2025-11-18 16:11:14,054 - INFO - 0 missing reviews detected and cleaned.
2025-11-18 16:11:16,815 - INFO - 16153 duplicated reviews detected and cleaned.
2025-11-18 16:17:04,046 - INFO - Numerical numbers converted to string numbers.
2025-11-18 16:52:19,842 - INFO -  Special characters removed.
Language detection: 100%|██████████| 6974127/6974127 [2:10:13<00:00, 892.54it/s]  
2025-11-18 19:30:50,642 - INFO - 7251 reviews are potentially not in english.
Translating non-English reviews: 100%|█████████▉| 7268/7269 [08:34<00:00, 14.13it/s]
2025-11-18 19:39:52,756 - INFO - 7251 have been translated in english.
2025-11-18 19:40:00,207 - INFO - Cleaned Dataframe saved at ..\data\processed\data_clean\data_yelp_reviews_cleaned.csv


In [101]:
if __name__ == "__main__":
     preprocess_pipeline("../data/original/dataset/data_airline_reviews_1.csv",
                         "review",
                         "../data/processed/data_clean/data_airline_reviews_1_cleaned.csv")

2025-11-18 19:40:00,354 - INFO - DataFrame data_airline_reviews_1 loaded : 3701 rows x 20 columns
2025-11-18 19:40:00,407 - INFO - 0 missing reviews detected and cleaned.
2025-11-18 19:40:00,459 - INFO - 9 duplicated reviews detected and cleaned.
2025-11-18 19:40:00,743 - INFO - Numerical numbers converted to string numbers.
2025-11-18 19:40:02,299 - INFO -  Special characters removed.
Language detection: 100%|██████████| 3692/3692 [00:05<00:00, 700.73it/s]
2025-11-18 19:40:08,586 - INFO - 0 reviews are potentially not in english.
Translating non-English reviews: 0it [00:00, ?it/s]
2025-11-18 19:40:08,603 - INFO - 0 have been translated in english.
2025-11-18 19:40:08,613 - INFO - Cleaned Dataframe saved at ../data/processed/data_clean/data_airline_reviews_1_cleaned.csv
